# Vehicle Search Agent API Testing

Interactive notebook for testing the API endpoints.

**Prerequisites:** Make sure the API server is running:
```bash
python api/server.py
api documentation: http://localhost:8000/docs
```

In [ ]:
import requests
import json
from pprint import pprint

BASE_URL = "http://localhost:8000"

: 

## 1. Health Check

In [ ]:
response = requests.get(f"{BASE_URL}/")
print(f"Status: {response.status_code}")
pprint(response.json())

## 2. Start a Conversation

In [ ]:
# Send first message
response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": "I want a Jeep around $30k in Colorado"
    }
)

data = response.json()
session_id = data.get('session_id')

print(f"Session ID: {session_id}")
print(f"\nAgent Response:")
print(data.get('response'))
print(f"\nVehicles Found: {len(data.get('vehicles', []))}")

# NEW: Display quick replies and suggested follow-ups
print("\n" + "="*70)
print("INTERACTIVE ELEMENTS:")
print("="*70)

quick_replies = data.get('quick_replies')
if quick_replies:
    print(f"\n💬 Quick Replies (click to answer):")
    for i, reply in enumerate(quick_replies, 1):
        print(f"   [{i}] {reply}")
else:
    print(f"\n💬 Quick Replies: None (no direct question asked)")

suggested_followups = data.get('suggested_followups', [])
if suggested_followups:
    print(f"\n💡 Suggested Follow-ups (conversation starters):")
    for i, followup in enumerate(suggested_followups, 1):
        print(f"   [{i}] {followup}")

## 3. View Filters

In [ ]:
print("Current Filters:")
pprint(data.get('filters'))

print("\nInferred Preferences:")
pprint(data.get('preferences'))

## 4. View Vehicle Listings

In [ ]:
vehicles = data.get('vehicles', [])
for i, vehicle in enumerate(vehicles[:5], 1):  # Show top 5
    v = vehicle.get('vehicle', {})
    listing = vehicle.get('retailListing', {})
    history = vehicle.get('history', {})
    photos = vehicle.get('photos')

    print(f"\n{'='*80}")
    print(f"#{i}. {v.get('year')} {v.get('make')} {v.get('model')} {v.get('trim', '')}")
    print(f"{'='*80}")

    # Price and basic info
    print(f"💰 Price:        ${listing.get('price', 0):,}")
    print(f"📏 Mileage:      {listing.get('miles', 0):,} miles")
    print(f"📍 Location:     {listing.get('city')}, {listing.get('state')} {listing.get('zip')}")
    print(f"🏢 Dealer:       {listing.get('dealer')}")

    # Vehicle details
    print(f"\n🚗 Vehicle Details:")
    print(f"   • Vehicle Details Page: {listing.get('vdp', 'N/A')}")
    print(f"   • Body Style:    {v.get('bodyStyle', 'N/A')}")
    print(f"   • Drivetrain:    {v.get('drivetrain', 'N/A')}")
    print(f"   • Engine:        {v.get('engine', 'N/A')}")
    print(f"   • Transmission:  {v.get('transmission', 'N/A')}")
    print(f"   • Exterior:      {v.get('exteriorColor', 'N/A')}")
    print(f"   • Interior:      {v.get('interiorColor', 'N/A')}")
    print(f"   • Doors:         {v.get('doors', 'N/A')}")
    print(f"   • Seats:         {v.get('seats', 'N/A')}")

    # History
    print(f"\n📋 Vehicle History:")
    print(f"   • Accidents:     {'Yes (' + str(history.get('accidentCount', 0)) + ')' if
history.get('accidents') else 'No'}")
    print(f"   • Owners:        {history.get('ownerCount', 'N/A')}")
    print(f"   • Usage:         {history.get('usageType', 'N/A')}")

    # Photos
    if photos and photos.get('retail'):
        photo_urls = photos.get('retail', [])
        print(f"\n📸 Photos:       {len(photo_urls)} available (cached)")
        print(f"   • First photo:   {photo_urls[0][:60]}...")
    else:
        print(f"\n📸 Photos:       No photos available")

    # Links
    print(f"\n🔗 Links:")
    print(f"   • VIN:           {v.get('vin')}")
    print(f"   • Listing:       {listing.get('vdp', 'N/A')[:70]}...")
    if listing.get('carfaxUrl'):
        print(f"   • CarFax:        {listing.get('carfaxUrl')[:70]}...")

print(f"\n{'='*80}")
print(f"Total vehicles: {len(vehicles)}")

## 5. Continue Conversation (Analytical Mode)

In [ ]:
# Ask about a specific vehicle
response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": "Tell me more about #1",
        "session_id": session_id
    }
)

data = response.json()

print(f"\nAgent Response:")
print(data.get('response'))

# NEW: Display interactive elements
print("\n" + "="*70)
print("INTERACTIVE ELEMENTS:")
print("="*70)

quick_replies = data.get('quick_replies')
if quick_replies:
    print(f"\n💬 Quick Replies:")
    for i, reply in enumerate(quick_replies, 1):
        print(f"   [{i}] {reply}")
else:
    print(f"\n💬 Quick Replies: None")

suggested_followups = data.get('suggested_followups', [])
if suggested_followups:
    print(f"\n💡 Suggested Follow-ups:")
    for i, followup in enumerate(suggested_followups, 1):
        print(f"   [{i}] {followup}")

## 6. Get Session State

In [ ]:
response = requests.get(f"{BASE_URL}/session/{session_id}")
session_data = response.json()

print("Session State:")
print(f"  Filters: {session_data.get('filters')}")
print(f"  Preferences: {session_data.get('preferences')}")
print(f"  Vehicles: {len(session_data.get('vehicles', []))}")
print(f"  Conversation Length: {len(session_data.get('conversation_history', []))} messages")

## 7. View Conversation History

In [ ]:
print("Conversation History:")
print("=" * 70)

for msg in session_data.get('conversation_history', []):
    role = msg.get('role', 'unknown').upper()
    content = msg.get('content', '')
    
    if role == 'USER':
        print(f"\n👤 User: {content}")
    else:
        print(f"\n🤖 Agent: {content[:200]}...")  # Truncate long responses
    print("-" * 70)

## 8. Get Vehicle Details by VIN

In [ ]:
# Get VIN from first vehicle and test Auto.dev API directly
import os
from dotenv import load_dotenv

load_dotenv()

if vehicles:
    vin = vehicles[0].get('vehicle', {}).get('vin')
    
    if vin:
        print(f"Testing Auto.dev API for VIN: {vin}\n")
        print("=" * 70)
        
        # Get Auto.dev API key
        autodev_api_key = os.getenv('AUTODEV_API_KEY')
        
        if not autodev_api_key:
            print("❌ AUTODEV_API_KEY not found in environment")
        else:
            headers = {
                "Authorization": f"Bearer {autodev_api_key}",
                "Content-Type": "application/json"
            }
            
            # Get vehicle listing details
            print("\n1. Vehicle Listing Details:")
            listing_response = requests.get(
                f"https://api.auto.dev/listings/{vin}",
                headers=headers
            )
            
            if listing_response.status_code == 200:
                listing_data = listing_response.json()
                print("✅ Listing found")
                pprint(listing_data)
            else:
                print(f"❌ Error {listing_response.status_code}: {listing_response.text}")
            
            # Get vehicle photos
            print("\n2. Vehicle Photos:")
            photos_response = requests.get(
                f"https://api.auto.dev/photos/{vin}",
                headers=headers
            )
            
            if photos_response.status_code == 200:
                photos_data = photos_response.json()
                retail_photos = photos_data.get('data', {}).get('retail', [])
                
                print(f"✅ Found {len(retail_photos)} photos")
                
                if retail_photos:
                    print("\nFirst 5 photo URLs:")
                    for i, url in enumerate(retail_photos[:5], 1):
                        print(f"  {i}. {url}")
            else:
                print(f"❌ Error {photos_response.status_code}: {photos_response.text}")
    else:
        print("No VIN found in first vehicle")
else:
    print("No vehicles available to query")

## 9. Custom Message (Try Your Own!)

In [ ]:
# Try your own message
custom_message = "Show me some vehicles now?"  

response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": custom_message,
        "session_id": session_id
    }
)

data = response.json()
print(f"Your Message: {custom_message}")
print(f"\nAgent Response:")
print(data.get('response'))

## 10. Reset Session

In [ ]:
response = requests.post(
    f"{BASE_URL}/session/reset",
    json={"session_id": session_id}
)

print("Session reset:")
pprint(response.json())

## 11. List All Active Sessions

In [ ]:
response = requests.get(f"{BASE_URL}/sessions")
sessions_data = response.json()

print(f"Active Sessions: {sessions_data.get('active_sessions')}")
print(f"Session IDs: {sessions_data.get('session_ids')}")

## 12. Test Event Tracking - Log User Interactions

In [ ]:
# Simulate user interactions with the recommendation list

# Example 1: User views vehicle details
if vehicles and len(vehicles) > 0:
    first_vehicle = vehicles[0]
    vin = first_vehicle.get('vehicle', {}).get('vin')
    
    print("📝 Logging event: User clicked to view vehicle #1")
    response = requests.post(
        f"{BASE_URL}/session/{session_id}/event",
        json={
            "event_type": "vehicle_click",
            "data": {
                "vin": vin,
                "vehicle_index": 0,
                "year": first_vehicle.get('vehicle', {}).get('year'),
                "make": first_vehicle.get('vehicle', {}).get('make'),
                "model": first_vehicle.get('vehicle', {}).get('model')
            }
        }
    )
    
    if response.status_code == 200:
        result = response.json()
        print(f"✅ Event logged successfully!")
        print(f"   Event ID: {result.get('event_id')}")
        print(f"   Timestamp: {result.get('timestamp')}")
    else:
        print(f"❌ Error: {response.status_code} - {response.text}")

# Example 2: User views photos
print("\n📝 Logging event: User viewed photos")
response = requests.post(
    f"{BASE_URL}/session/{session_id}/event",
    json={
        "event_type": "photo_view",
        "data": {
            "vin": vin,
            "vehicle_index": 0,
            "photo_count": len(first_vehicle.get('photos', {}).get('retail', [])) if first_vehicle.get('photos') else 0
        }
    }
)

if response.status_code == 200:
    print(f"✅ Photo view event logged!")
    print(f"   Event ID: {response.json().get('event_id')}")

# Example 3: User clicks CarFax link
print("\n📝 Logging event: User clicked CarFax link")
response = requests.post(
    f"{BASE_URL}/session/{session_id}/event",
    json={
        "event_type": "link_click",
        "data": {
            "vin": vin,
            "link_type": "carfax",
            "url": first_vehicle.get('retailListing', {}).get('carfaxUrl')
        }
    }
)

if response.status_code == 200:
    print(f"✅ Link click event logged!")

# Example 4: Log a custom event (e.g., user scrolled to position 5)
print("\n📝 Logging event: User scrolled to vehicle #5")
response = requests.post(
    f"{BASE_URL}/session/{session_id}/event",
    json={
        "event_type": "scroll_position",
        "data": {
            "max_index_viewed": 4,
            "total_vehicles": len(vehicles)
        }
    }
)

if response.status_code == 200:
    print(f"✅ Scroll event logged!")

print("\n" + "="*70)

## 13. Retrieve All Logged Events

In [ ]:
# Get all events for the session
print("🔍 Retrieving all interaction events...\n")

response = requests.get(f"{BASE_URL}/session/{session_id}/events")

if response.status_code == 200:
    events_data = response.json()
    
    print(f"📊 Total Events: {events_data.get('total')}")
    print(f"Session ID: {events_data.get('session_id')}\n")
    print("="*70)
    
    for i, event in enumerate(events_data.get('events', []), 1):
        print(f"\n📌 Event #{i}")
        print(f"   Type:      {event.get('event_type')}")
        print(f"   Timestamp: {event.get('timestamp')}")
        print(f"   Data:")
        
        for key, value in event.get('data', {}).items():
            print(f"      • {key}: {value}")
        
        print("-"*70)
    
    # Filter by event type
    print("\n\n🔍 Filtering: Only 'vehicle_click' events\n")
    response = requests.get(f"{BASE_URL}/session/{session_id}/events?event_type=vehicle_click")
    
    if response.status_code == 200:
        filtered_data = response.json()
        print(f"Found {filtered_data.get('total')} vehicle click events")
        for event in filtered_data.get('events', []):
            vin = event.get('data', {}).get('vin')
            print(f"   • VIN: {vin} at {event.get('timestamp')}")
else:
    print(f"❌ Error: {response.status_code} - {response.text}")

## 14. Test Exploration Mode

The system now includes an exploration phase where the agent asks questions like a real salesperson before making recommendations.

In [ ]:
# Create a new session for exploration testing
response = requests.post(f"{BASE_URL}/session/reset")
exploration_session_id = response.json().get('session_id')

print(f"New Session ID: {exploration_session_id}")
print("\n" + "="*70)
print("EXPLORATION MODE TEST")
print("="*70)
print("\nThe agent will ask questions before showing vehicles.")
print("Watch the 'exploration_mode' field in responses.\n")

# Turn 1: Initial greeting
print("\n[Turn 1] User: Hi, I'm looking for a car")
response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": "Hi, I'm looking for a car",
        "session_id": exploration_session_id
    }
)

data = response.json()
print(f"\nExploration Mode: {data.get('exploration_mode')}")
print(f"Vehicles Count: {len(data.get('vehicles', []))}")
print(f"\n🤖 Agent: {data.get('response')}")
print("\n" + "-"*70)

In [ ]:
# Turn 2: Answer first question
print("[Turn 2] User: It's my first car, for commuting to work")
response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": "It's my first car, for commuting to work",
        "session_id": exploration_session_id
    }
)

data = response.json()
print(f"\nExploration Mode: {data.get('exploration_mode')}")
print(f"Vehicles Count: {len(data.get('vehicles', []))}")
print(f"Exploration Insights: {data.get('exploration_insights')}")
print(f"\n🤖 Agent: {data.get('response')}")
print("\n" + "-"*70)

In [ ]:
# Turn 3-4: Continue exploration
messages = [
    "Budget is around $30k, reliability is important",
    "I live in the city, about 20 miles daily commute",
    "Yes, show me some options"
]

for i, msg in enumerate(messages, 3):
    print(f"[Turn {i}] User: {msg}")
    response = requests.post(
        f"{BASE_URL}/chat",
        json={
            "message": msg,
            "session_id": exploration_session_id
        }
    )
    
    data = response.json()
    print(f"\nExploration Mode: {data.get('exploration_mode')}")
    print(f"Vehicles Count: {len(data.get('vehicles', []))}")
    
    if data.get('exploration_mode') == 'complete':
        print("✅ Exploration complete! Showing recommendations.")
    
    print(f"\n🤖 Agent: {data.get('response')[:200]}...")
    print("\n" + "-"*70 + "\n")

## Key Observations About Exploration Mode

**What to look for:**
1. **exploration_mode**: 
   - `"active"` = Agent is still asking questions
   - `"complete"` = Agent is ready to show vehicles
   
2. **vehicles**: Empty list (`[]`) during exploration, populated after completion

3. **exploration_insights**: Contains what the agent learned:
   - `use_cases`: How they'll use the vehicle
   - `lifestyle_notes`: Family, lifestyle context
   - `must_haves`: Critical requirements
   - `current_situation`: Their current vehicle situation

**Configuration:**
- Set `MAX_EXPLORATION_QUESTIONS` in `.env` to control question limit (default: 6)
- Agent will automatically transition after max questions reached
- User can force early transition by saying "show me cars" or similar

## 15. 🎯 NEW: Streaming Chat API with Real-Time Progress

**This section shows how to use the `/chat/stream` endpoint with actual runnable code.**

The streaming endpoint sends progress updates in real-time as the agent processes your request.

In [ ]:
import requests
import json

def stream_chat(message, session_id=None):
    """
    Stream chat with real-time progress updates.
    
    Args:
        message: Your question or request
        session_id: Optional session ID for conversation continuity
        
    Returns:
        Final response data
    """
    url = f"{BASE_URL}/chat/stream"
    
    payload = {"message": message}
    if session_id:
        payload["session_id"] = session_id
    
    print(f"📨 Sending: '{message}'")
    print("\n" + "="*70)
    print("PROGRESS UPDATES:")
    print("="*70 + "\n")
    
    response = requests.post(
        url,
        json=payload,
        stream=True
    )
    
    event_type = None
    progress_count = 0
    final_data = None
    
    # Process Server-Sent Events
    for line in response.iter_lines(decode_unicode=True):
        if not line:
            continue
        
        line = line.strip()
        
        if line.startswith('event:'):
            event_type = line.split(':', 1)[1].strip()
            
        elif line.startswith('data:'):
            if not event_type:
                continue
                
            data_json = line.split(':', 1)[1].strip()
            data = json.loads(data_json)
            
            if event_type == 'progress':
                progress_count += 1
                description = data.get('description', '')
                status = data.get('status', '')
                
                # Show progress with emoji
                emoji = "⏳" if status == "in_progress" else "✅"
                print(f"{emoji} [{progress_count}] {description}")
                
            elif event_type == 'complete':
                final_data = data
                
            elif event_type == 'error':
                print(f"\n❌ ERROR: {data.get('error')}")
                return None
            
            # Reset for next event
            event_type = None
    
    print("\n" + "="*70)
    print("✨ COMPLETE!")
    print("="*70)
    
    return final_data

### Comparison: Regular vs Streaming API

Let's compare the two endpoints:

In [ ]:
import time

message = "Show me Honda Civic"

print("="*70)
print("COMPARISON: Regular vs Streaming")
print("="*70)

# 1. Regular endpoint (no progress)
print("\n1️⃣ Regular /chat endpoint (no progress updates):")
print("-" * 70)
start = time.time()

regular_response = requests.post(
    f"{BASE_URL}/chat",
    json={"message": message}
)

regular_time = time.time() - start
regular_data = regular_response.json()

print(f"⏱️  Time: {regular_time:.2f}s")
print(f"✅ Response received (no progress updates during wait)")
print(f"🚗 Vehicles: {len(regular_data.get('vehicles', []))}")

# 2. Streaming endpoint (with progress)
print(f"\n2️⃣ Streaming /chat/stream endpoint (with progress):")
print("-" * 70)
start = time.time()

streaming_result = stream_chat(message)

streaming_time = time.time() - start

print(f"\n⏱️  Time: {streaming_time:.2f}s")
print(f"✅ Same result, but with real-time progress updates!")
print(f"🚗 Vehicles: {len(streaming_result.get('vehicles', []))}")

print("\n" + "="*70)
print("CONCLUSION:")
print("="*70)
print("• Both endpoints take similar time")
print("• Streaming provides better UX with progress updates")
print("• Use /chat/stream for web UI to show loading states")
print("="*70)

In [ ]:
# 🆕 Example: Compare Top 3 Vehicles

## Step 1: Get Honda Accord listings
print("Step 1: Get Honda Accord listings")
print("="*70)

response = requests.post(
    f"{BASE_URL}/chat",
    json={"message": "Show me 2024 Honda Accord vehicles"}
)

data = response.json()
session_id = data['session_id']

print(f"✅ Found {len(data['vehicles'])} vehicles\n")

# Show top 3
for i, v in enumerate(data['vehicles'][:3], 1):
    vehicle = v.get('vehicle', {})
    retail = v.get('retailListing', {})
    print(f"#{i}: {vehicle['year']} {vehicle['make']} {vehicle['model']} {vehicle.get('trim', '')} - ${retail['price']:,} - {retail['city']}, {retail['state']}")

## Step 2: Compare top 3
print("\n" + "="*70)
print("Step 2: Compare top 3")
print("="*70)

response = requests.post(
    f"{BASE_URL}/chat",
    json={
        "message": "compare top 3",
        "session_id": session_id
    }
)

data = response.json()

print(f"\n🤖 Agent: {data['response']}\n")

# Display comparison table
if data.get('comparison_table'):
    import pandas as pd
    
    table = data['comparison_table']
    
    # Convert to pandas DataFrame for nice display
    df_data = {}
    for i, header in enumerate(table['headers']):
        df_data[header] = [row[i] for row in table['rows']]
    
    df = pd.DataFrame(df_data)
    
    print("📊 Comparison Table:")
    display(df)
else:
    print("⚠️ No comparison table generated")
